# Sum of Structured Tensors
```
Copyright 2022 National Technology & Engineering Solutions of Sandia,
LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS, the
U.S. Government retains certain rights in this software.
```

When certain operations are performed on a tensor which is formed as a sum of tensors, it can be beneficial to avoid explicitly forming the sum. For example, if a tensor is formed as a sum of a low rank tensor and a sparse tensor, the structure of the summands can make storage, decomposition and operations with other tensors significantly more efficient. A `sumtensor` exploits this structure. Here we explain the basics of defining and using sumtensors.

In [ ]:
import pyttb as ttb
import numpy as np
import pickle

In [ ]:
def estimate_mem_usage(variable) -> int:
    """
    Python variables contain references to memory.
    Quickly estimate memory usage of custom types.
    """
    return len(pickle.dumps(variable))

## Creating sumtensors
A sumtensor `T` can only be delared as a sum of same-shaped tensors T1, T2,...,TN. The summand tensors are stored internally, which define the "parts" of the `sumtensor`. The parts of a `sumtensor` can be (dense) tensors (`tensor`), sparse tensors (` sptensor`), Kruskal tensors (`ktensor`), or Tucker tensors (`ttensor`). An example of the use of the sumtensor constructor follows.

In [ ]:
T1 = ttb.tenones((3, 3, 3))
T2 = ttb.sptensor(
    subs=np.array([[0, 0, 0], [1, 1, 1], [2, 2, 1], [1, 0, 0]]),
    vals=np.ones((4, 1)),
    shape=(3, 3, 3),
)
T = ttb.sumtensor([T1, T2])
print(T)

## A Magnitude Example
For large-scale problems, the `sumtensor` class may make the difference as to whether or not a tensor can be stored in memory. Consider the following example, where $\mathcal{T}$ is of size $1000 x 1000 x 1000$, formed from the sum of a `ktensor` and an `sptensor`.

In [ ]:
np.random.seed(0)
X1 = np.random.rand(50, 3)
X2 = np.random.rand(50, 3)
X3 = np.random.rand(50, 3)
K = ttb.ktensor([X1, X2, X3], np.ones((3,)), copy=False)
S = ttb.sptenrand((50, 50, 50), 1e-100)

ST = ttb.sumtensor([K, S])
TT = ST.full()
print(
    f"Size of sumtensor: {estimate_mem_usage(ST)}\n"
    f"Size of tensor: {estimate_mem_usage(TT)}"
)

## Further examples of the sumtensor constructor
We can declare an empty sumtensor, with no parts.

In [ ]:
P = ttb.sumtensor()
print(P)

`sumtensor` also supports a copy constructor.

In [ ]:
S = P.copy()
print(S)

## Ndims and shape for dimensions of a sumtensor
For a given `sumtensor`, `ndims` returns the number of modes and `shape` returns the shape in each dimension of the `sumtensor`.

In [ ]:
print(f"Ndims: {T.ndims}\n" f"Shape: {T.shape}")

## Use full to convert sumtensor to a dense tensor
The `full` method can convert all the parts of a `sumtensor` to a dense tensor. Note that for large tensors, this can use a large amount of memory to expand then sum the parts.

In [ ]:
print(T.full())

## Use double to convert to a numpy array
The `double` method can convert the parts of a `sumtensor` to a dense numpy array. Similar warnings for memory usages as `full`.

In [ ]:
print(T.double())

## Matricized Khatri-Rao product of a sumtensor
The `mttkrp` method computes the Khatri-Rao product of a matricized tensor and `sumtensor`. The required arguments are:
* A list of matrices (or a `ktensor`)
* A mode n

The list of matrices must consist of m matrices, where m is the number of modes in the `sumtensor`. The number of columns in all matrices should be the same and the number of rows of matrix i should match the dimension of the `sumtensor` shape in mode i. For more details see the documentation of `tensor.mttkrp`.

In [ ]:
matrices = [np.eye(3), np.ones((3, 3)), np.random.rand(3, 3)]
n = 1
T.mttkrp(matrices, n)

## Innerproducts of sumtensors
The `innerprod` method computes the inner product of a `sumtensors` parts with other tensor types.

In [ ]:
S = ttb.sptensor(
    subs=np.array([[0, 0, 0], [1, 1, 1], [2, 2, 1], [1, 0, 0]]),
    vals=np.ones((4, 1)),
    shape=(3, 3, 3),
)
T.innerprod(S)

## Norm compatibility interface
The `norm` method just returns 0 and issues a warning. Norm cannot be distributed, but some algorithms access the norm for verbose details.

In [ ]:
T.norm()

## Use CP-ALS with sumtensor
One of the primary motivations for defining the `sumtensor` class is for efficient decomposition. In particular, when trying to find a CP decomposition of a tensor using alternating least squares, the subproblems can be efficiently created and solved using mttkrp and innerprod. Both of these operations can be performed more efficiently by exploiting extra structure in the tensors which form the sum, so the performance of `cp_als` is also improved. Consider the following example, where a cp_als is run on a sumtensor.

In [ ]:
result, _, _ = ttb.cp_als(T, 2, maxiters=10)
print(result)

It follows that in cases where $\mathcal{T}$ is too large for its full expansion to be stored in memory, we may still be able find a CP decomposition by exploiting the sumtensor structure.

_Note_ that the fit returned by `cp_als` is not correct for `sumtensor`, because the norm operation is not supported.

## Addition with sumtensors
Sumtensors can be added to any other type of tensor. The result is a new `sumtensor` with the tensor appended to the parts of the original `sumtensor`. Note that the tensor is always appended, despite the order of the operation.

In [ ]:
# Equivalent additions despite the order
print(f"T+S:\n{T+S}\n")
print(f"S+T:\n{S+T}")

## Accessing sumtensor parts
Subscripted reference can be used to access individual parts of the `sumtensor`.

In [ ]:
print(f"Part 0:\n{T.parts[0]}\n\n" f"Part 1:\n{T.parts[1]}")